In [1]:
#%load_ext autoreload
#%autoreload 2

import os; print(os.getcwd())
import socket; print(socket.gethostname())
import truncator
from glob import glob
import Bio

/net/scratch/ajasja/projects/truncator/R4_crystal_nMS_Y2H
dig23


In [19]:
def pdb_to_fasta(pdb_name, fasta_name=None):
    import pymol
    from pymol import cmd
    if fasta_name is None:
        fasta_name = pdb_name.replace('.pdb.gz', '.fasta').replace('.pdb', '.fasta')
    cmd.delete('all')
    cmd.load(pdb_name)
    print(fasta_name)
    cmd.save(fasta_name)

In [20]:
out_dir = "out/R3/!to_order/2018.03.01"
out_dir_pdbs = out_dir+"/pdbs"
truncator.make_dirs(out_dir_pdbs)
pdbs = glob("out/R3/!final/*.pdb")
pdbs = [pdb for pdb in pdbs if "__" in os.path.basename(pdb)]
for pdb in pdbs:
    !cp -v {pdb} {out_dir_pdbs}

'out/R3/!final/ZCON_13__numH4.5__from-17.52__to22.91__ccBC-DA__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-10.pdb' -> 'out/R3/!to_order/2018.03.01/pdbs/ZCON_13__numH4.5__from-17.52__to22.91__ccBC-DA__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-10.pdb'
'out/R3/!final/ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-10.pdb' -> 'out/R3/!to_order/2018.03.01/pdbs/ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-10.pdb'
'out/R3/!final/ZCON_127__numH4.5__from-20.77__to21.47__ccDA-BC__llr2,5__ar1-3,0__ar2-3,0__rms0.5__ch-10.pdb' -> 'out/R3/!to_order/2018.03.01/pdbs/ZCON_127__numH4.5__from-20.77__to21.47__ccDA-BC__llr2,5__ar1-3,0__ar2-3,0__rms0.5__ch-10.pdb'
'out/R3/!final/ZCON_37__numH4.5__from-21.21__to22.24__ccAB-CD__llr2,5__ar1-3,0__ar2-2,0__rms0.5__ch-10.pdb' -> 'out/R3/!to_order/2018.03.01/pdbs/ZCON_37__numH4.5__from-21.21__to22.24__ccAB-CD__llr2,5__ar1-3,0__ar2-2,0__rms0.5__ch-10.pdb'
'out/R3/!final/ZCON_127__numH4.5__from-20.77__

In [21]:
from glob import glob
pdbs = glob(out_dir_pdbs+"/*.pdb")
pdbs = [pdb for pdb in pdbs if "__" in os.path.basename(pdb)]
for pdb in pdbs:
    #pdb_to_fasta("struct/ALSD01_AB_DHR08_BA_B_0001_0001.rn_0001_0004_cut.pdb")
    basename = pdb.replace('.pdb.gz', '').replace('.pdb', '')
    out_name = basename + "_raw.fasta"
    #print(out_name)
    pdb_to_fasta(pdb, out_name)

 PyMOL not running, entering library mode (experimental)
out/R3/!to_order/2018.03.01/pdbs/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__ccAD-CB__llr2,5__ar1-2,0__ar2-1,0__rms0.5__ch-10_raw.fasta
out/R3/!to_order/2018.03.01/pdbs/ZCON_37__numH4.5__from-21.21__to22.24__ccAB-CD__llr2,5__ar1-3,0__ar2-2,0__rms0.5__ch-10_raw.fasta
out/R3/!to_order/2018.03.01/pdbs/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__ccC-BAD__llr2,5__ar1-2,0__ar2-1,0__rms0.5__ch-10_raw.fasta
out/R3/!to_order/2018.03.01/pdbs/ZCON_13__numH4.5__from-17.52__to22.91__ccBC-DA__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-10_raw.fasta
out/R3/!to_order/2018.03.01/pdbs/ZCON_13__numH4.5__from-17.52__to22.91__ccA-BCD__llr2,5__ar1-1,0__ar2-1,0__rms0.5__ch-10_raw.fasta
out/R3/!to_order/2018.03.01/pdbs/ZCON_127__numH4.5__from-20.77__to21.47__ccC-DAB__llr2,4__ar1-2,0__ar2-3,0__rms0.5__ch-10_raw.fasta
out/R3/!to_order/2018.03.01/pdbs/ZCON_37__numH4.5__from-21.21__to22.24__ccA-DCB__llr2,5__ar1-2,0__ar2-3,0__rms0.5__ch-10_raw.fasta
out/R3/!to_

In [22]:
def seq_charge(seq):
    charge = 0
    AACharge = {"C":-.045,"D":-1,"E":-1,"H":.091,
                "K":1,"R":1,"Y":-.001}
    for aa in seq:
        charge += AACharge.get(aa,0)
    return charge
seq_charge('DEMKKVMEALKKAVELAKKNNDDEVAREIERAAKEIVEALRENNSDEMAKVMLALAKAVLLAAKNNDDEVAREIARAAAEIVEALRENNSDEMAKVMLALAKAVLLAAKNNDDEVAREIARAADEIVRALRENNSDEQAKKMLKLAKEVLDAAKNNDDDTARALAVVAAYVVLLDRHEKAV')

-7.91

In [42]:
from Bio import SeqIO
N=0;
fastas = sorted(glob(out_dir_pdbs+"/*_raw.fasta"))
with open(f'{out_dir}/FINAL_with_tags.fasta','w+') as f:
    for fasta in fastas:
        N=N+1
        for i, record in enumerate(SeqIO.parse(fasta, "fasta")):
            #or I could grep name for topology 
            n_chains = round(len(record.seq)/30)
            if i==0: #chain A
                chain="A"
                print(f">mALb{N}_{n_chains}{chain} | (His-GFP_pET29b+(XhoI))__{record.id}", file=f)
                print("SGDS DESGEDGS GSSGSG ENLYFQG S", file=f)
                print(record.seq, file=f)
                print("G", file=f)
                print("", file=f)
            if i==1: #chain A
                chain="B"
                print(f">mALb{N}_{n_chains}{chain} |(pET29b+)__{record.id}", file=f)
                print("GSSHHHHHHS SGDS GLNDIFEAQKIEWHE DESGEDGS GSSGSG ENLYFQG S", file=f)
                print(record.seq, file=f)
                print("G**", file=f)
                print("", file=f)

In [55]:
def domesticate(name, seq, out_name):
    !/home/rdkibler/.conda/envs/domesticator/bin/python /home/rdkibler/projects/domesticator/domesticator.py \
    --protein_sequence {seq} \
    --avoid_restriction_sites NdeI,XhoI \
    --output_mode fasta \
    --avoid_patterns CATATG,CTCGAG,AGGAGG,TAAGGAG,GCTGGTGG,TTTTTT,AAAAAAA,ATCTGTT,GGRGGT,MAGGTRAG,YYYYNTAGG \
    --sequence_name {name} \
    --avoid_kmer_boost 20 \
    --output_filename {out_name}.fasta > {out_name}.log
    
for i, record in enumerate(SeqIO.parse(f'{out_dir}/FINAL_with_tags.fasta', "fasta")):
    print(record.id)
    out_name = out_dir+'/domesticator/'+record.id
    domesticate(record.id, record.seq, out_name)

mALb1_1A
location:   0%|                                | 0/1 [00:00<?, ?it/s, now=0-171]
                                                                                
location:   0%|                                  | 0/9 [00:00<?, ?it/s, now=0-3]
                                                                                
location:   0%|                                  | 0/9 [00:00<?, ?it/s, now=0-3]
mALb1_3B
location:   0%|                                | 0/1 [00:00<?, ?it/s, now=0-411]
                                                                                
location:   0%|                                 | 0/31 [00:00<?, ?it/s, now=6-9]
                                                                                
location: 100%|███████████████████████| 31/31 [00:00<00:00, 220.55it/s, now=6-9]
mALb2_2A
location:   0%|                                 | 0/15 [00:00<?, ?it/s, now=0-3]
                                                                                
l

In [64]:
fastas = sorted(glob(out_dir+'/domesticator/*.fasta'))
reverse={}
for fasta in fastas:
    for record in SeqIO.parse(fasta, "fasta"):
        print(record.id.replace("  <unknown description>",''))
        reverse[record.id.replace("  <unknown description>",'')]=record

mALb1_1A
mALb1_3B
mALb2_2A
mALb2_2B
mALb3_1A
mALb3_3B
mALb4_2A
mALb4_2B
mALb5_1A
mALb5_3B
mALb6_1A
mALb6_3B
mALb7_1A
mALb7_3B
mALb8_2A
mALb8_2B


In [66]:
originals={}
for record in SeqIO.parse(f'{out_dir}/FINAL_with_tags.fasta', "fasta"):
    originals[record.id]=record

In [72]:
for key in originals.keys():
    print(key)
    orig=originals[key].seq
    rev=reverse[key].translate(to_stop=False).seq
    if orig!=rev:
        pritn("SEQUENCE NOT THE SAME: WARNING WARNING")

mALb1_1A
mALb1_3B
mALb2_2A
mALb2_2B
mALb3_1A
mALb3_3B
mALb4_2A
mALb4_2B
mALb5_1A
mALb5_3B
mALb6_1A
mALb6_3B
mALb7_1A
mALb7_3B
mALb8_2A
mALb8_2B


In [76]:
with open(f'{out_dir}/FINAL_with_tags.DNA.fasta','w+') as f:
    for key in originals.keys():
        #print(key)
        print(">"+reverse[key].id, file=f)
        print(reverse[key].seq, file=f)
        print("", file=f)


In [78]:
with open(f'{out_dir}/FINAL_with_tags.DNA.his-GFP_XhoI.fasta','w+') as f:
    for key in originals.keys():
        #print(key)
        if '(His-GFP_pET29b+(XhoI))' in originals[key].description:
            print(">"+reverse[key].id, file=f)
            print(reverse[key].seq, file=f)
            print("", file=f)
            
with open(f'{out_dir}/FINAL_with_tags.DNA.pet29b+.fasta','w+') as f:
    for key in originals.keys():
        #print(key)
        if '(pET29b+)' in originals[key].description:
            print(">"+reverse[key].id, file=f)
            print(reverse[key].seq, file=f)
            print("", file=f)

In [85]:
##Get full sequences
with open(f'{out_dir}/FINAL_FULL_SEQ.fasta','w+') as f:
    for record in SeqIO.parse(f'{out_dir}/FINAL_with_tags.fasta', "fasta"):
        #print(record.name)
        #print(record.seq)
        if '(His-GFP_pET29b+(XhoI))' in record.description:

            print(">"+record.description, file=f)
            print("MHHHHHHGSSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELY", file=f)
            print(record.seq, file=f)
            print("**", file=f)
            print("", file=f)

        #print(key)
        if '(pET29b+)' in record.description:
            print(">"+record.description, file=f)
            print("M", file=f)
            print(record.seq, file=f)
            print("", file=f)

mALb1_1A
SGDSDESGEDGSGSSGSGENLYFQGSDREEILNSQRELLELIEELLKTQKKLEEKQG
mALb1_3B
GSSHHHHHHSSGDSGLNDIFEAQKIEWHEDESGEDGSGSSGSGENLYFQGSSEEEKLVKRSSQIANDIREKSNQNVEQGGSDELAKLSQTLVELLRTLVEASEKGEDKNELSRLHEELEKAVKEVRETHEREEKG**
mALb2_2A
SGDSDESGEDGSGSSGSGENLYFQGSSEVQELVRKSKEIANTITERSNQNVEKGKSEELAKLSQELVKLLKELVEESEETTEG
mALb2_2B
GSSHHHHHHSSGDSGLNDIFEAQKIEWHEDESGEDGSGSSGSGENLYFQGSDKEEQLKKLHERLRQAVERVRETHERAQERGDVTEEVAKSQEDLLKLIEELLQTQEQLNERMG**
mALb3_1A
SGDSDESGEDGSGSSGSGENLYFQGSGILERQQEIIEEAKEIHRRQKEILREDDG
mALb3_3B
GSSHHHHHHSSGDSGLNDIFEAQKIEWHEDESGEDGSGSSGSGENLYFQGSSRLEESLKLLERLEELSQESAELLEKGEDDERAKEATKAHKKQEEIIRKAQELHKQLKENKDKKILRTIEELSKESEELLEQILTLSEQQG**
mALb4_2A
SGDSDESGEDGSGSSGSGENLYFQGSSELDESLNLLRELEELSKQSAELLREGDQDSWATRAKQAHDEQNEIIKRAQELHKQEKG
mALb4_2B
GSSHHHHHHSSGDSGLNDIFEAQKIEWHEDESGEDGSGSSGSGENLYFQGSSITEEIEELSERSQKLLEEILELSERSKDNEIIQEQTEIIRRASNIHKRQKDILESTKG**
mALb5_1A
SGDSDESGEDGSGSSGSGENLYFQGSSDEERSKELQRESRKLLKEMAEIMENG
mALb5_3B
GSSHHHHHHSSGDSGLNDIFEAQKIEWHEDESGEDGSGSSGSGENLYFQGSDKEKRLQK

In [2]:
out_dir = "out/R3/!to_order/2018.03.01"

In [4]:
!python2 /home/ajasja/scripts/get_protparam.py {out_dir}/FINAL_FULL_SEQ.fasta

>mALb1_1A
MHHHHHHGSSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYSGDSDESGEDGSGSSGSGENLYFQGSDREEILNSQRELLELIEELLKTQKKLEEKQG**
()
len:304
# aa			302
Molecular weight	33805.56
Theoretical pI		5.35
Ext. coefficient	20525
Abs 0.1%		0.607, assuming all pairs of Cys residues form cystines
Ext. coefficient	20400
Abs 0.1%		0.603, assuming all Cys residues are reduced
Monoisotopic mass	33784.679154
Charge			-15.644


>mALb1_3B
MGSSHHHHHHSSGDSGLNDIFEAQKIEWHEDESGEDGSGSSGSGENLYFQGSSEEEKLVKRSSQIANDIREKSNQNVEQGGSDELAKLSQTLVELLRTLVEASEKGEDKNELSRLHEELEKAVKEVRETHEREEKG**
()
len:138
# aa			136
Molecular weight	15193.22
Theoretical pI		4.88
Ext. coefficient	 6990
Abs 0.1%		0.460
Monoisotopic mass	15184.25113
Charge			-15.182


>mALb2_2A
MHHHHHHGSSKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYP

In [56]:
!{sys.executable} -c 'import codon_harmony; codon_harmony.runner()'

/bin/sh: {sys.executable}: command not found


In [ ]:
import sys
!cd 'out/R3/!to_order/2018.03.01'; {sys.executable} -c 'import codon_harmony; codon_harmony.runner()' --input FINAL_with_tags_no_stop.fasta --output FINAL_with_tags.DNA.fasta \
--restriction-enzymes NdeI XhoI BamHI \
--host-threshold 0.05 \
--cycles 50 --inner-cycles 50

In [35]:
!for f in fasta/*.fasta; do echo /home/brunette/src/psipred3.21/runpsipred_csbuild_single $f; done

/bin/sh: -c: line 0: syntax error near unexpected token `;'
/bin/sh: -c: line 0: `for f in fasta/*.fasta; do echo /home/brunette/src/psipred3.21/runpsipred_csbuild_single <_io.TextIOWrapper name='fasta/ASLD01_only_longhelix_B.fasta' mode='w+' encoding='UTF-8'>; done'


In [ ]:
!python2 /home/ajasja/scripts/get_protparam.py FINAL.clean.fasta  | tee raw_fasta.log